In [296]:
import pandas as pd
import re

In [297]:
messages  =["""
BULK
 Item Id   Number       Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks  
 37144805  NULL         ESTIMATE  0.00      KG   0         0.00      DUB       DUB       EDI       B         5         false     NULL 
ULD
 Item Id   ULD       Number    Airline   Special   Status    Weight         Pieces    Volume    Origin    Onl.Sta.  Dest.     Category  Position  Confirmed  Remarks  
 37137862  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         11        false     NULL 
 37137860  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         12        false     NULL 
 37137858  AKH       NULL      MN                  ESTIMATE  351.00    KG   27        0.00      DUB       DUB       EDI       B         41        false     NULL 
 37137861  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         13        false     NULL 
 37137857  AKH       NULL      MN                  ESTIMATE  455.00    KG   35        0.00      DUB       DUB       EDI       B         42        false     NULL 
STATUS FUEL 2 AIRCRAFT_CONFIG 1 EZFW 1 CARGO_TRANSFER 1 OFP 4 CABIN_CONFIG 1 AUTO_MODE_ACTIVE 1 AUTOMATION_STARTED 0 BAG_LOAD_ITEMS_GEN 1 EZFW_COUNTER 1 REGISTRATION 1 REGISTRATION_CHANGE 1 FUEL_ORDER 2",f506e2277d8b1ff7,Saved
"""]

In [298]:
def metaFunction(messages):
    data = []
    for message in messages:

        uld = findType("ULD", message)
        bulk = findType("BULK", message)
        bulk = list(set(bulk) - set(uld))

        bulk = bulk[1:]
        bulk = [bulk[0]] +  bulk[2:]
        bulk = bulk[:-1]
        bulk = [bulk[0]] + bulk[2:]
        bulk = bulk

        uldIndices = findStartingCharactersIndices("ULD")
        bulkIndices = findStartingCharactersIndices("BULK")
        
        uldData = fetchData(uld, uldIndices)
        bulkData = fetchData(bulk, bulkIndices)
        combined = uldData + bulkData
        data.append(combined)
    return pd.DataFrame(data[0])

In [299]:
def findType(type, string):
    match = re.search(fr"{type}", string)
    substring = ""
    if match:
        substring += string[match.start():]
    substring = substring.split("\n")
    if type == "ULD":
        return substring[1:-2]
    return substring

def findStartingCharactersIndices(type):
    dummy_row = None
    if type == "ULD":
        dummy_row = "37137862  AKH       NULL      MN        NULL      ESTIMATE  88.00     KG   0         0.00      DUB       DUB       EDI       X         11        false     NULL"
    else:
        dummy_row = "37144805  NULL         ESTIMATE  0.00      KG   0         0.00      DUB       DUB       EDI       B         5         false     NULL"
    dummy_row = dummy_row.replace("KG", "")
    words = dummy_row.split()
    indices = []
    current_index = 0

    # Iterate through each word
    for word in words:
        # Get the first character of the word
        first_char = word[0]
        # Find the index of the first character in the original string
        index = dummy_row.index(first_char, current_index)
        # Append the index and character to the lists
        indices.append(index)
        # Update the current index to search for the next word
        current_index = index + len(word)
    return indices

def fetchData(substring, indices):
    columns = re.sub(r'\s+', ',', substring[0].lstrip().replace("Item Id", "ItemID"))[:-1].split(",")
    data = []
    for line in substring[1:]:
        line = line.lstrip()
        line = line.replace("KG","")
        dummy_row_list = list(line)
        for index in indices:
            if dummy_row_list[index] == " ":
                # Replace the character and the following 3 characters with "NULL"
                dummy_row_list[index:index+4] = "NULL"

        # Convert the list back to a string
        line = ''.join(dummy_row_list)

        line = re.sub(r'\s+', ',', line)
        line = line.split(",")
        dictionary = dict(zip(columns, line))
        data.append(dictionary)
    return data

In [300]:
df = metaFunction(messages)
df

,ItemID,ULD,Number,Airline,Special,Status,Weight,Pieces,Volume,Origin,Onl.Sta.,Dest.,Category,Position,Confirmed,Remarks
0,37137862,AKH,NULL,MN,NULL,ESTIMATE,88.00,0,0.00,DUB,DUB,EDI,X,11,false,NULL
1,37137860,AKH,NULL,MN,NULL,ESTIMATE,88.00,0,0.00,DUB,DUB,EDI,X,12,false,NULL
2,37137858,AKH,NULL,MN,NULL,ESTIMATE,351.00,27,0.00,DUB,DUB,EDI,B,41,false,NULL
3,37137861,AKH,NULL,MN,NULL,ESTIMATE,88.00,0,0.00,DUB,DUB,EDI,X,13,false,NULL
4,37137857,AKH,NULL,MN,NULL,ESTIMATE,455.00,35,0.00,DUB,DUB,EDI,B,42,false,NULL
5,37144805,NaN,NULL,NaN,NaN,ESTIMATE,0.00,0,0.00,DUB,DUB,EDI,B,5,false,NULL
